# CISC 372 Final Project

Student name: Liangjin LIN (20164732), Cheuk Him CHAU (20378374), Masood Ahmed (20379714), 


The three research questions are:
1. Is there a gap between the efficiency of using a single Session-Based Recommendation (SBR) algorithm and Session-Based Recommendation (SBR) along with multi-task learning?
2. How sensitive are the parameters to these algorithms?
3. Is there a gap between the efficiency of the algorithms
mentioned involving cold-start users?



Findings on data:
1. "QK-video", "QK-article", "QB-video", "QB-article" are raw datasets.
2. There is a dataset called "sbr_data_1M", which is used for SBR in the original paper. We can use this dataset for our project.

## Data Exploration

In [1]:
# import necessary libraries
# from google.colab import drive
import pandas as pd
#import tensorflow as tf
import math

In [2]:
# Do not use this, upload the data here is very slow

# upload file from local system
# from google.colab import files
# uploades = files.upload()

# Use this instead, the dataset is uploaded to the shared google drive
# drive.mount('/content/gdrive')

In [3]:
# !ls "./gdrive/MyDrive/CISC351 Advance Data Analytics/Dataset"
# !ls "./gdrive/MyDrive/"

In [4]:
data_path = "./Tenrec/sbr_data_1M.csv"
sbr_data = pd.read_csv(data_path)
sbr_data.head()

c:\Users\wilso\anaconda3\envs\cisc351\lib\site-packages\IPython\core\interactiveshell.py:3186: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,user_id,item_id,click,follow,like,share,video_category,watching_times,gender,age
0,1,1,1,0,0,0,1,1,1,4
1,1,2,1,0,0,0,1,1,1,4
2,1,3,1,0,0,0,0,1,1,4
3,1,80936,1,0,0,0,1,1,1,4
4,1,781,1,0,0,0,1,1,1,4


In [5]:
df = sbr_data.loc[:25599]
df.groupby("user_id")["item_id"].count()

user_id
1       71
2       15
3       37
4       27
5       53
      ... 
557    160
558     59
559     91
560     19
561      1
Name: item_id, Length: 561, dtype: int64

In [6]:
# Preprocess the data

import argparse
import time
import csv
import pickle
import operator
import datetime
import os

# Read the data
print("-- Starting @ %ss" % datetime.datetime.now())
sess_clicks = {}

ctr = 0

for _, data in df.iterrows():
    sessid = data['user_id']

    item = data["item_id"]

    if sessid in sess_clicks:
        sess_clicks[sessid] += [item]
    else:
        sess_clicks[sessid] = [item]
    ctr += 1

print("-- Reading data @ %ss" % datetime.datetime.now())

# Filter out sessions with length less than 10
for s in list(sess_clicks):
    if len(sess_clicks[s]) < 10:
        del sess_clicks[s]

# Set the max session length to 30
for key, ls in sess_clicks.items():
  if len(ls) > 30:
    sess_clicks[key] = ls[-30:]

# Transform users' ids and interactions into a 2d list
user_interactions = [ls for _, ls in sess_clicks.items()]
user_ids = [id for id, _ in sess_clicks.items()]

# splitting the data into training and testing sets
n_10pct = len(user_interactions) // 10
test_sess = user_interactions[:n_10pct]
train_sess = user_interactions[n_10pct+1:]
test_ids = user_ids[:n_10pct]
test_ids = user_ids[n_10pct+1:]

# Count the number of unique items in training set
items = set()
ctr = 1
for seq in train_sess:
    for s in seq:
        if s not in items:
            items.add(s)
            ctr +=1
print(ctr)

# If we need to split the validation set out by hand
# test_sess = user_interactions[:n_10pct]
# valid_sess = user_interactions[n_10pct+1:2*n_10pct]
# train_sess = user_interactions[2*n_10pct+1:]
# test_ids = user_ids[:n_10pct]
# valid_ids = user_ids[n_10pct+1:2*n_10pct]
# test_ids = user_ids[n_10pct+1:]

def process_seqs(in_seqs):
    out_seqs = []
    labs = []
    for seq in in_seqs:
        for i in range(1, len(seq)):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [seq[:-i]]
    return out_seqs, labs


train_seqs, train_labs = process_seqs(train_sess)
test_seqs, test_labs = process_seqs(test_sess)
train = (train_seqs, train_labs)
test = (test_seqs, test_labs)

-- Starting @ 2023-04-11 12:31:32.955105s
-- Reading data @ 2023-04-11 12:31:34.569106s
9414


SR-GNN model

In [7]:
#!/usr/bin/env python36
# -*- coding: utf-8 -*-
"""
Created on July, 2018
@author: Tangrizzly
"""

import datetime
import math
import numpy as np
import torch
from torch import nn
from torch.nn import Module, Parameter
import torch.nn.functional as F


class GNN(Module):
    def __init__(self, hidden_size, step=1):
        super(GNN, self).__init__()
        self.step = step
        self.hidden_size = hidden_size
        self.input_size = hidden_size * 2
        self.gate_size = 3 * hidden_size
        self.w_ih = Parameter(torch.Tensor(self.gate_size, self.input_size))
        self.w_hh = Parameter(torch.Tensor(self.gate_size, self.hidden_size))
        self.b_ih = Parameter(torch.Tensor(self.gate_size))
        self.b_hh = Parameter(torch.Tensor(self.gate_size))
        self.b_iah = Parameter(torch.Tensor(self.hidden_size))
        self.b_oah = Parameter(torch.Tensor(self.hidden_size))

        self.linear_edge_in = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_edge_out = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_edge_f = nn.Linear(self.hidden_size, self.hidden_size, bias=True)

    def GNNCell(self, A, hidden):
        input_in = torch.matmul(A[:, :, :A.shape[1]], self.linear_edge_in(hidden)) + self.b_iah
        input_out = torch.matmul(A[:, :, A.shape[1]: 2 * A.shape[1]], self.linear_edge_out(hidden)) + self.b_oah
        inputs = torch.cat([input_in, input_out], 2)
        gi = F.linear(inputs, self.w_ih, self.b_ih)
        gh = F.linear(hidden, self.w_hh, self.b_hh)
        i_r, i_i, i_n = gi.chunk(3, 2)
        h_r, h_i, h_n = gh.chunk(3, 2)
        resetgate = torch.sigmoid(i_r + h_r)
        inputgate = torch.sigmoid(i_i + h_i)
        newgate = torch.tanh(i_n + resetgate * h_n)
        hy = newgate + inputgate * (hidden - newgate)
        return hy

    def forward(self, A, hidden):
        for i in range(self.step):
            hidden = self.GNNCell(A, hidden)
        return hidden


class SessionGraph(Module):
    def __init__(self, opt, n_node):
        super(SessionGraph, self).__init__()
        self.hidden_size = opt["hiddenSize"]
        self.n_node = n_node
        self.batch_size = opt["batchSize"]
        self.nonhybrid = opt["nonhybrid"]
        self.embedding = nn.Embedding(self.n_node, self.hidden_size)
        self.gnn = GNN(self.hidden_size, step=opt["step"])
        self.linear_one = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_two = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_three = nn.Linear(self.hidden_size, 1, bias=False)
        self.linear_transform = nn.Linear(self.hidden_size * 2, self.hidden_size, bias=True)
        self.loss_function = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=opt["lr"], weight_decay=opt["l2"])
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=opt["lr_dc_step"], gamma=opt["lr_dc"])
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)

    def compute_scores(self, hidden, mask):
        ht = hidden[torch.arange(mask.shape[0]).long(), torch.sum(mask, 1) - 1]  # batch_size x latent_size
        q1 = self.linear_one(ht).view(ht.shape[0], 1, ht.shape[1])  # batch_size x 1 x latent_size
        q2 = self.linear_two(hidden)  # batch_size x seq_length x latent_size
        alpha = self.linear_three(torch.sigmoid(q1 + q2))
        a = torch.sum(alpha * hidden * mask.view(mask.shape[0], -1, 1).float(), 1)
        if not self.nonhybrid:
            a = self.linear_transform(torch.cat([a, ht], 1))
        b = self.embedding.weight[1:]  # n_nodes x latent_size
        scores = torch.matmul(a, b.transpose(1, 0))
        return scores

    def forward(self, inputs, A):
        hidden = self.embedding(inputs)
        hidden = self.gnn(A, hidden)
        return hidden


def trans_to_cuda(variable):
    if torch.cuda.is_available():
        return variable.cuda()
    else:
        return variable


def trans_to_cpu(variable):
    if torch.cuda.is_available():
        return variable.cpu()
    else:
        return variable


def forward(model, i, data):
    alias_inputs, A, items, mask, targets = data.get_slice(i)
    alias_inputs = trans_to_cuda(torch.Tensor(alias_inputs).long())
    items = trans_to_cuda(torch.Tensor(items).long())
    A = trans_to_cuda(torch.Tensor(A).float())
    mask = trans_to_cuda(torch.Tensor(mask).long())
    print(items.size())
    print(A.size())
    hidden = model(items, A)
    get = lambda i: hidden[i][alias_inputs[i]]
    seq_hidden = torch.stack([get(i) for i in torch.arange(len(alias_inputs)).long()])
    return targets, model.compute_scores(seq_hidden, mask)


def train_test(model, train_data, test_data):
    model.scheduler.step()
    print('start training: ', datetime.datetime.now())
    model.train()
    total_loss = 0.0
    slices = train_data.generate_batch(model.batch_size)
    for i, j in zip(slices, np.arange(len(slices))):
        model.optimizer.zero_grad()
        targets, scores = forward(model, i, train_data)
        targets = trans_to_cuda(torch.Tensor(targets).long())
        loss = model.loss_function(scores, targets - 1)
        loss.backward()
        model.optimizer.step()
        total_loss += loss
        if j % int(len(slices) / 5 + 1) == 0:
            print('[%d/%d] Loss: %.4f' % (j, len(slices), loss.item()))
    print('\tLoss:\t%.3f' % total_loss)

    print('start predicting: ', datetime.datetime.now())
    model.eval()
    hit, mrr = [], []
    slices = test_data.generate_batch(model.batch_size)
    for i in slices:
        targets, scores = forward(model, i, test_data)
        sub_scores = scores.topk(20)[1]
        sub_scores = trans_to_cpu(sub_scores).detach().numpy()
        for score, target, mask in zip(sub_scores, targets, test_data.mask):
            hit.append(np.isin(target - 1, score))
            if len(np.where(score == target - 1)[0]) == 0:
                mrr.append(0)
            else:
                mrr.append(1 / (np.where(score == target - 1)[0][0] + 1))
    hit = np.mean(hit) * 100
    mrr = np.mean(mrr) * 100
    return hit, mrr

In [8]:
import networkx as nx
import numpy as np


def build_graph(train_data):
    graph = nx.DiGraph()
    for seq in train_data:
        for i in range(len(seq) - 1):
            if graph.get_edge_data(seq[i], seq[i + 1]) is None:
                weight = 1
            else:
                weight = graph.get_edge_data(seq[i], seq[i + 1])['weight'] + 1
            graph.add_edge(seq[i], seq[i + 1], weight=weight)
    for node in graph.nodes:
        sum = 0
        for j, i in graph.in_edges(node):
            sum += graph.get_edge_data(j, i)['weight']
        if sum != 0:
            for j, i in graph.in_edges(i):
                graph.add_edge(j, i, weight=graph.get_edge_data(j, i)['weight'] / sum)
    return graph


def data_masks(all_usr_pois, item_tail):
    us_lens = [len(upois) for upois in all_usr_pois]
    len_max = max(us_lens)
    us_pois = [upois + item_tail * (len_max - le) for upois, le in zip(all_usr_pois, us_lens)]
    us_msks = [[1] * le + [0] * (len_max - le) for le in us_lens]
    return us_pois, us_msks, len_max


def split_validation(train_set, valid_portion):
    train_set_x, train_set_y = train_set
    n_samples = len(train_set_x)
    sidx = np.arange(n_samples, dtype='int32')
    np.random.shuffle(sidx)
    n_train = int(np.round(n_samples * (1. - valid_portion)))
    valid_set_x = [train_set_x[s] for s in sidx[n_train:]]
    valid_set_y = [train_set_y[s] for s in sidx[n_train:]]
    train_set_x = [train_set_x[s] for s in sidx[:n_train]]
    train_set_y = [train_set_y[s] for s in sidx[:n_train]]

    return (train_set_x, train_set_y), (valid_set_x, valid_set_y)


class Data():
    def __init__(self, data, shuffle=False, graph=None):
        inputs = data[0]
        inputs, mask, len_max = data_masks(inputs, [0])
        self.inputs = np.asarray(inputs)
        self.mask = np.asarray(mask)
        self.len_max = len_max
        self.targets = np.asarray(data[1])
        self.length = len(inputs)
        self.shuffle = shuffle
        self.graph = graph

    def generate_batch(self, batch_size):
        if self.shuffle:
            shuffled_arg = np.arange(self.length)
            np.random.shuffle(shuffled_arg)
            self.inputs = self.inputs[shuffled_arg]
            self.mask = self.mask[shuffled_arg]
            self.targets = self.targets[shuffled_arg]
        n_batch = int(self.length / batch_size)
        if self.length % batch_size != 0:
            n_batch += 1
        slices = np.split(np.arange(n_batch * batch_size), n_batch)
        slices[-1] = slices[-1][:(self.length - batch_size * (n_batch - 1))]
        return slices

    def get_slice(self, i):
        inputs, mask, targets = self.inputs[i], self.mask[i], self.targets[i]
        items, n_node, A, alias_inputs = [], [], [], []
        for u_input in inputs:
            n_node.append(len(np.unique(u_input)))
        max_n_node = np.max(n_node)
        for u_input in inputs:
            node = np.unique(u_input)
            items.append(node.tolist() + (max_n_node - len(node)) * [0])
            u_A = np.zeros((max_n_node, max_n_node))
            for i in np.arange(len(u_input) - 1):
                if u_input[i + 1] == 0:
                    break
                u = np.where(node == u_input[i])[0][0]
                v = np.where(node == u_input[i + 1])[0][0]
                u_A[u][v] = 1
            u_sum_in = np.sum(u_A, 0)
            u_sum_in[np.where(u_sum_in == 0)] = 1
            u_A_in = np.divide(u_A, u_sum_in)
            u_sum_out = np.sum(u_A, 1)
            u_sum_out[np.where(u_sum_out == 0)] = 1
            u_A_out = np.divide(u_A.transpose(), u_sum_out)
            u_A = np.concatenate([u_A_in, u_A_out]).transpose()
            A.append(u_A)
            alias_inputs.append([np.where(node == i)[0][0] for i in u_input])
        return alias_inputs, A, items, mask, targets

In [9]:
# Pad 0 into those lists with length less than 30
# for row in user_interactions:
#     row += [0] * (30 - len(row))

In [10]:
# import pandas as pd
# from collections import defaultdict

# processedData = pd.DataFrame( dict([ (k, pd.Series(v)) for k,v in sess_clicks.items()]) )

In [11]:
# Training

import os
import torch.backends.cudnn as cudnn
cudnn.benchmark = False
cudnn.deterministic = True
torch.set_default_tensor_type(torch.DoubleTensor)
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'

opt = {"batchSize":16, 
"hiddenSize" : 256,
"epoch" : 5,
"lr" : 0.001,
"lr_dc" : 0.1,
"lr_dc_step" : 3,
"l2" : 1e-5,
"step" : 1,
"patience" : 10,
"nonhybrid" : "store_true",
"validation" : "store_true",
"valid_portion" : 0.1}
n_node = 9414

train_data = Data(train, shuffle=True)
test_data = Data(test, shuffle=False)

model = trans_to_cuda(SessionGraph(opt, n_node))

start = time.time()
best_result = [0, 0]
best_epoch = [0, 0]
bad_counter = 0
for epoch in range(opt["epoch"]):
    print('-------------------------------------------------------')
    print('epoch: ', epoch)
    hit, mrr = train_test(model, train_data, test_data)
    flag = 0
    if hit >= best_result[0]:
        best_result[0] = hit
        best_epoch[0] = epoch
        flag = 1
    if mrr >= best_result[1]:
        best_result[1] = mrr
        best_epoch[1] = epoch
        flag = 1
    print('Best Result:')
    print('\tRecall@20:\t%.4f\tMMR@20:\t%.4f\tEpoch:\t%d,\t%d'% (best_result[0], best_result[1], best_epoch[0], best_epoch[1]))
    bad_counter += 1 - flag
    if bad_counter >= opt.patience:
        break
print('-------------------------------------------------------')
end = time.time()
print("Run time: %f s" % (end - start))

-------------------------------------------------------
epoch:  0
start training:  2023-04-11 12:31:36.421835
torch.Size([16, 28])
torch.Size([16, 28, 56])


c:\Users\wilso\anaconda3\envs\cisc351\lib\site-packages\torch\optim\lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
c:\Users\wilso\anaconda3\envs\cisc351\lib\site-packages\ipykernel_launcher.py:114: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:233.)


RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`

In [ ]:
print(torch.cuda.max_memory_allocated())

In [ ]:
# Select GPU device
device = torch.device("cuda")

# Get current memory usage
print(torch.cuda.memory_allocated(device=device))

In [ ]:
from __future__ import division
import numpy as np
import pickle
import argparse
import datetime

method = "ggnn"
epoch = 5
batchSize = 256
hiddenSize = 256
penalty = 0.00001
learningRate = 0.001
step = 1
lr_dc = 0.1
lr_dc_step = 3
nonhybrid = 'store_true'
n_node = 4000


train_data = Data(train, sub_graph=True, method=method, shuffle=True)
test_data = Data(test, sub_graph=True, method=method, shuffle=False)
model = GGNN(hidden_size=hiddenSize, out_size=hiddenSize, batch_size=batchSize, n_node=n_node,
                 lr=learningRate, l2=penalty,  step=step, decay=lr_dc_step * len(train_data.inputs) / batchSize, lr_dc=lr_dc,
                 nonhybrid=nonhybrid)

best_result = [0, 0]
best_epoch = [0, 0]
for epoch in range(epoch):
    print('epoch: ', epoch, '===========================================')
    slices = train_data.generate_batch(model.batch_size)
    fetches = [model.opt, model.loss_train, model.global_step]
    print('start training: ', datetime.datetime.now())
    loss_ = []
    for i, j in zip(slices, np.arange(len(slices))):
        print("slices: ", slices)
        print(i)
        print(j)
        adj_in, adj_out, alias, item, mask, targets = train_data.get_slice(i)
        _, loss, _ = model.run(fetches, targets, item, adj_in, adj_out, alias,  mask)
        loss_.append(loss)
    loss = np.mean(loss_)
    slices = test_data.generate_batch(model.batch_size)
    print('start predicting: ', datetime.datetime.now())
    hit, mrr, test_loss_ = [], [],[]
    for i, j in zip(slices, np.arange(len(slices))):
        adj_in, adj_out, alias, item, mask, targets = test_data.get_slice(i)
        scores, test_loss = model.run([model.score_test, model.loss_test], targets, item, adj_in, adj_out, alias,  mask)
        test_loss_.append(test_loss)
        index = np.argsort(scores, 1)[:, -20:]
        for score, target in zip(index, targets):
            hit.append(np.isin(target - 1, score))
            if len(np.where(score == target - 1)[0]) == 0:
                mrr.append(0)
            else:
                mrr.append(1 / (20-np.where(score == target - 1)[0][0]))
    hit = np.mean(hit)*100
    mrr = np.mean(mrr)*100
    test_loss = np.mean(test_loss_)
    if hit >= best_result[0]:
        best_result[0] = hit
        best_epoch[0] = epoch
    if mrr >= best_result[1]:
        best_result[1] = mrr
        best_epoch[1]=epoch
    print('train_loss:\t%.4f\ttest_loss:\t%4f\tRecall@20:\t%.4f\tMMR@20:\t%.4f\tEpoch:\t%d,\t%d'%
          (loss, test_loss, best_result[0], best_result[1], best_epoch[0], best_epoch[1]))

In [ ]:
train.length

In [ ]:
print(train_data.mask)
print(train_data.len_max)

print(train_data.inputs)
